In [1]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00


In [3]:
import torch
print(torch.__version__)

2.2.1+cu121


In [4]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.1+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.2.1+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.1/508.1 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.9/213.9 kB 28.9 MB/s eta 0:00:00


In [5]:
from torch.nn import Linear
from torch.nn.functional import dropout
from torch_geometric.nn import SAGEConv, global_mean_pool, GCNConv
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import random_split

In [115]:
data = TUDataset(root='../data/TUDataset/DD', name='DD').to('cuda')

In [7]:
len(data)

1178

In [8]:
data[0]

Data(edge_index=[2, 1798], x=[327, 89], y=[1])

In [128]:
train_set, val_set, test_set = random_split(data, [0.7, 0.2, 0.1])

In [129]:
train_loader = DataLoader(train_set, batch_size=2, shuffle=True)

In [130]:
val_loader = DataLoader(val_set, batch_size=2, shuffle = True)

In [131]:
def train(model, train_loader, val_loader, optimizer, criterion, epochs=5):
    for epoch in range(epochs):
        for i, batch in enumerate(train_loader):
            model.train()

            out = model(batch.x, batch.edge_index, batch.batch)

            loss = criterion(out, batch.y)
            loss.backward()
            train_loss = loss.item()

            optimizer.step()
            optimizer.zero_grad()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Loss: {train_loss:.4f}')

        for i, batch in enumerate(val_loader):
            model.eval()

            out = model(batch.x, batch.edge_index, batch.batch)

            loss = criterion(out, batch.y)
            val_loss = loss.item()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Val Loss: {val_loss:.4f}')

In [32]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, num_classes):
        super(GraphSAGE, self).__init__()

        self.conv1 = SAGEConv((-1, -1), 64)
        self.conv2 = SAGEConv((-1, -1), 128)
        self.conv3 = SAGEConv((-1, -1), 64)

        self.linear1 = Linear(64, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = dropout(x, p=0.5, training=self.training)
        x = self.linear1(x)

        return x

#Exercise-1 - Graph Classification with GraphSAGE

In [33]:
model = GraphSAGE(num_classes=2).to('cuda')

In [132]:
optimizer = Adam(model.parameters(), lr=0.01)
criterion = CrossEntropyLoss()

In [35]:
train(model, train_loader, val_loader, optimizer, criterion, 10)

Epoch: 000, Step: 000, Loss: 0.6793
Epoch: 000, Step: 001, Loss: 0.6424
Epoch: 000, Step: 002, Loss: 0.8263
Epoch: 000, Step: 003, Loss: 0.6683
Epoch: 000, Step: 004, Loss: 0.6780
Epoch: 000, Step: 005, Loss: 0.8388
Epoch: 000, Step: 006, Loss: 0.8534
Epoch: 000, Step: 007, Loss: 0.6929
Epoch: 000, Step: 008, Loss: 0.7579
Epoch: 000, Step: 009, Loss: 0.7165
Epoch: 000, Step: 010, Loss: 0.6709
Epoch: 000, Step: 011, Loss: 0.6723
Epoch: 000, Step: 012, Loss: 0.6494
Epoch: 000, Step: 013, Loss: 0.6230
Epoch: 000, Step: 014, Loss: 0.7667
Epoch: 000, Step: 015, Loss: 0.6585
Epoch: 000, Step: 016, Loss: 0.6459
Epoch: 000, Step: 017, Loss: 0.6147
Epoch: 000, Step: 018, Loss: 0.6827
Epoch: 000, Step: 019, Loss: 0.6173
Epoch: 000, Step: 020, Loss: 0.7201
Epoch: 000, Step: 021, Loss: 0.7323
Epoch: 000, Step: 022, Loss: 0.9054
Epoch: 000, Step: 023, Loss: 0.6792
Epoch: 000, Step: 024, Loss: 0.7193
Epoch: 000, Step: 025, Loss: 0.5558
Epoch: 000, Step: 026, Loss: 0.8269
Epoch: 000, Step: 027, Loss:

In [17]:
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
from torch_geometric.data import Batch

In [133]:
from sklearn.metrics import f1_score

def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    predictions = []
    true_labels = []

    with torch.inference_mode():
        for i, batch in enumerate(test_loader):
            out = model(batch.x, batch.edge_index, batch.batch)

            pred = out.argmax(dim=1)

            predictions.extend(pred.cpu().numpy())
            true_labels.extend(batch.y.cpu().numpy())

    return predictions, true_labels

In [134]:
test_loader = DataLoader(test_set, batch_size=2, shuffle=True)

In [20]:
from sklearn.metrics import classification_report

In [38]:
test_y, preds = test(model, test_loader, criterion)

In [39]:
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.95      0.67      0.78       105
           1       0.19      0.67      0.29        12

    accuracy                           0.67       117
   macro avg       0.57      0.67      0.54       117
weighted avg       0.87      0.67      0.73       117



#Exercise-2 - Graph Classification with GCN

In [141]:
class GCN(torch.nn.Module):
    def __init__(self, num_classes):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(-1, 64)
        self.conv2 = GCNConv(-1, 128)
        self.conv3 = GCNConv(-1, 64)

        self.linear1 = Linear(64, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = dropout(x, p=0.5, training=self.training)
        x = self.linear1(x)

        return x

In [142]:
model2 = GCN(num_classes=2).to('cuda')

In [143]:
train(model2, train_loader, val_loader, optimizer, criterion, 10)

Streaming output truncated to the last 5000 lines.
Epoch: 000, Step: 310, Loss: 0.7282
Epoch: 000, Step: 311, Loss: 0.7222
Epoch: 000, Step: 312, Loss: 0.7235
Epoch: 000, Step: 313, Loss: 0.6974
Epoch: 000, Step: 314, Loss: 0.6926
Epoch: 000, Step: 315, Loss: 0.7252
Epoch: 000, Step: 316, Loss: 0.6961
Epoch: 000, Step: 317, Loss: 0.7192
Epoch: 000, Step: 318, Loss: 0.6871
Epoch: 000, Step: 319, Loss: 0.7259
Epoch: 000, Step: 320, Loss: 0.6903
Epoch: 000, Step: 321, Loss: 0.7234
Epoch: 000, Step: 322, Loss: 0.7238
Epoch: 000, Step: 323, Loss: 0.7315
Epoch: 000, Step: 324, Loss: 0.7231
Epoch: 000, Step: 325, Loss: 0.6902
Epoch: 000, Step: 326, Loss: 0.6610
Epoch: 000, Step: 327, Loss: 0.7237
Epoch: 000, Step: 328, Loss: 0.6920
Epoch: 000, Step: 329, Loss: 0.6955
Epoch: 000, Step: 330, Loss: 0.6981
Epoch: 000, Step: 331, Loss: 0.6976
Epoch: 000, Step: 332, Loss: 0.7266
Epoch: 000, Step: 333, Loss: 0.6908
Epoch: 000, Step: 334, Loss: 0.6681
Epoch: 000, Step: 335, Loss: 0.7253
Epoch: 000, S

In [144]:
test_y, preds = test(model2, test_loader, criterion)

In [140]:
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.95      0.57      0.72       110
           1       0.08      0.57      0.14         7

    accuracy                           0.57       117
   macro avg       0.52      0.57      0.43       117
weighted avg       0.90      0.57      0.68       117

